# Задание

**ЦЕЛЬ:** разработать систему, которая будет формировать описания покупателей на основе характеристик, представленных в файле
ㅤ
ㅤ
## Общие требования к программе

- код должен быть грамотно декомпозирован: каждая функция отвечает за свою конкретную задачу, дублирующиеся функции переиспользуются, а их код не повторяется;
- в коде отсутствуют глобальные переменные.
ㅤ
### Шаги
- [x]загрузить csv-файл;
- [x]выполнить парсинг;
- [x]преобразовать данные;
- [x]сформирвать текстовое описание по шаблону;
- [x]записать в единый txt-файл.
ㅤ
### Входные данные

строки из csv-файла, в которых есть следующие столбцы:
- name (ФИО);
- device_type (устройство, с которого выполнялась покупка);
- browser (браузер);
- sex (пол);
- age (возраст);
- bill (сумма чека);
- region (регион покупки).
ㅤ
### Выходные данные

строки в txt-файле - описание по шаблону:
Пользователь {name} {sex} пола, {age} лет(года) совершил(а) покупку на {bill} у.е. с {device_type} браузера {browser}. Регион, из которго совершалась покупка: {region}.

### особенности, которые были учтены
- возрастозависимость (лет/года/год)
- тип устройства локализуется (мобильного/компьютерного)
- пол локализуется (мужского/женского)
- в csv-файле был age = 0.67 - посчитал за равным 0

In [ ]:
class Purchase:
    '''
        класс ПОКУПКА:
            customer - клиент
            browser - бразуер, в котором была совершена покупка
            bill - стоимость покупки
    '''

    def __init__(self,  args):
        self.customer = Customer(args['customer'])
        self.browser = Browser(args['browser'])
        self.bill = args['bill']

    def get(self) -> str:
        return f"Пользователь {self.customer.name['fullname']} {self.customer.sex['description']} пола, {self.customer.age['value']} {self.customer.age['description']} {'совершила' if self.customer.sex['value'] == 'female' else 'совершил'} покупку на {self.bill} у.е. с {self.browser.device_type['description']} браузера {self.browser.browser['value']}. Регион, из которго совершалась покупка: {self.browser.region['value']}.\n"


class Browser:
    '''
        класс БРАУЗЕР:
            browser - {value - название браузера}
            device_type - {value - название девайса, description - мобильного/компьютерного/-}
            region - {value - где была покупка}
    '''

    def __init__(self, browser_):
        self.browser = {
            'value': browser_['browser']
        }
        self.device_type = {
            'value': browser_['device_type'],
            'description': self.__get_device_description(browser_['device_type'])
        }
        self.region = {
            'value': browser_['region']
        }

    def __get_device_description(self, device_type):
        match(device_type):
            case 'mobile' | 'tablet': return 'мобильного'
            case 'laptop' | 'desktop': return 'компьютерного'
            case _: return '-'


class Customer:
    '''
        класс КЛИЕНТ:
            name - {fullname - ФИО клиента}
            age - {value - возраст, description - год/года/лет}
            sex - {value - пол, description - мужского/женского}
    '''

    def __init__(self, customer_):
        self.name = {
            'fullname': customer_['name']
        }
        self.age = {
            'value': customer_['age'],
            'description': self.__get_age_description(customer_['age'])
        }
        self.sex = {
            'value': customer_['sex'],
            'description': self.__get_sex_description(customer_['sex'])
        }

    def __get_sex_description(self, sex):
        match(sex):
            case 'male': return 'мужского'
            case 'female': return 'женского'
            case _: return '-'

    def __get_age_description(self, age):
        if not (age):
            return 'лет'

        ld = age % 10  # последняя цифра
        if 1 < ld < 5:
            return 'года'
        if ld == 1:
            return 'год'
        return 'лет'


def get_fields(current_list):
    return {
        'customer': {'name': current_list[0], 'sex': current_list[3], 'age': int(float(current_list[4]))},
        'browser': {'browser': current_list[2], 'device_type': current_list[1], 'region': current_list[6]},
        'bill': int(current_list[5])
    }


# константы
FIN_PATH = 'web_clients_correct.csv'
FOUT_PATH = 'web_clients_descriptions.txt'
ENC = 'utf-8'

# чтение csv-файла, парсинг, запись в txt-файл
with open(FIN_PATH, 'r', encoding=ENC) as fin, open(FOUT_PATH, 'w', encoding=ENC) as fout:
    fin.readline()  # срезаем первую строку (шапку)
    for row in fin:
        current_fields = get_fields(row.strip().split(','))
        current_purchase = Purchase(current_fields)
        fout.write(current_purchase.get())